In [4]:
from IPython.parallel import Client
client = Client()

In [5]:
dview = client.direct_view()
len(client.ids)

3

In [6]:
all_engines = client[:]
all_engines

<DirectView [0, 1, 2]>

In [7]:
%%px

import os
import socket
print("This is running in process with pid {0} on host '{1}'.".format(
      os.getpid(), socket.gethostname()))

[stdout:0] This is running in process with pid 3226 on host 'master'.
[stdout:1] This is running in process with pid 2163 on host 'node001'.
[stdout:2] This is running in process with pid 2132 on host 'node002'.


In [8]:
def my_sum(a, b):
    return a + b

my_sum_apply_results = all_engines.apply(my_sum, 11, 31)
my_sum_apply_results
my_sum_apply_results.get()

[42, 42, 42]

In [10]:
%px a = 1

In [11]:
lv = client.load_balanced_view()

In [12]:
def slow_square(x):
    import time
    time.sleep(2)
    return x ** 2

In [13]:
result = lv.apply(slow_square, 4)
result
result.ready()
result.get()

16

In [15]:
results = lv.map(slow_square, [0, 1, 2, 3])
results
results.ready()
results.progress
for r in results:
    print (r)

0
1
4
9


In [16]:
%px import numpy as np

In [17]:
%%px --targets=-1  
# replace with one_engine_by_host.values()

# Cleanup any existing file from past session (necessary for windows)
import os
if os.path.exists('small.mmap'):
    os.unlink('small.mmap')

mm_w = np.memmap('small.mmap', shape=10, dtype=np.float32, mode='w+')
print(mm_w)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [18]:
%%px

mm_r = np.memmap('small.mmap', dtype=np.float32, mode='r+')
print(mm_r)

[stdout:0] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[stdout:1] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[stdout:2] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [19]:
%%px --targets=-1  
# replace with one_engine_by_host.values()

mm_w[0] = 42
print(mm_w)
print(mm_r)

[ 42.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
[ 42.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [21]:
%px print(mm_r)

[stdout:0] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[stdout:1] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[stdout:2] [ 42.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [22]:
%%px --targets=1

mm_r[1] = 43

In [23]:
%%px
print(mm_r)

[stdout:0] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[stdout:1] [  0.  43.   0.   0.   0.   0.   0.   0.   0.   0.]
[stdout:2] [ 42.   0.   0.   0.   0.   0.   0.   0.   0.   0.]


In [24]:
%%px
print("sum={0:.3}, mean={1:.3}, std={2:.3}".format(
    mm_r.sum(), np.mean(mm_r), np.std(mm_r)))

[stdout:0] sum=0.0, mean=0.0, std=0.0
[stdout:1] sum=43., mean=4.3, std=12.9
[stdout:2] sum=42., mean=4.2, std=12.6


In [29]:
with all_engines.sync_imports():
    import psutil

ImportError: No module named psutil

Traceback (most recent call last):
  File "/usr/lib/python2.7/logging/__init__.py", line 872, in emit
    stream.write(fs % msg)
IOError: [Errno 32] Broken pipe
Logged from file ipkernel.py, line 427


In [35]:
%px

!pip install psutil sgeadmin


sgeadmin is not in the sudoers file.  This incident will be reported.

[sudo] password for sgeadmin: 

In [25]:
%px

def get_host_free_memory(client):
    """Free memory on each host of the cluster in MB."""
    all_engines = client[:]
    def hostname():
        import socket
        return socket.gethostname()
    
    hostnames = all_engines.apply(hostname).get_dict()
    one_engine_per_host = dict((hostname, engine_id)
                               for engine_id, hostname
                               in hostnames.items())

    def host_free_memory():
        import psutil
        return psutil.virtual_memory().free / 1e6
    
    
    host_mem = client[one_engine_per_host.values()].apply(
        host_free_memory).get_dict()
    
    return dict((hostnames[eid], m) for eid, m in host_mem.items())

In [26]:
get_host_free_memory(client)

CompositeError: one or more exceptions from call to method: host_free_memory
[2:apply]: ImportError: No module named psutil
[1:apply]: ImportError: No module named psutil
[0:apply]: ImportError: No module named psutil

Traceback (most recent call last):
  File "/usr/lib/python2.7/logging/__init__.py", line 872, in emit
    stream.write(fs % msg)
IOError: [Errno 32] Broken pipe
Logged from file ipkernel.py, line 427
